In [4]:
import torch

x = torch.tensor(1.0)
y = torch.tensor(2.0)

# This is the parameter we want to optimize -> requires_grad=True
w = torch.tensor(1.0, requires_grad = True)

# forward pass to compute loss
y_predicted = w*x
loss = (y_predicted-y)**2
print(loss)



tensor(1., grad_fn=<PowBackward0>)


In [2]:
# backward pass to compute gradient dLoss/dw
loss.backward()
print(w.grad)

tensor(-2.)


In [3]:
# update weights
# next forward and backward pass...

# continue optimizing:
# update weights, this operation should not be part of the computational graph
with torch.no_grad():
    w -= 0.01 * w.grad
# don't forget to zero the gradients
w.grad.zero_()

# next forward and backward pass...

tensor(0.)

STEP-01
- prediction = manually
- gradient computations = manually
- Loss computation = manually
-  Parameter updates = manually

In [20]:
import numpy as np

X = np.array([1,2,3,4], dtype = np.float32)
Y = np.array([2,4,6,8], dtype = np.float32)

w = 0.0
def forward(x):
    return w*x

def loss(y,y_pred):
    return((y_pred - y)**2).mean()

def gradient(x,y,y_pred):
    return np.mean(2*x*(y_pred-y))

print(f'Prediction before training:f(5)={forward(5):.3f}')

#Training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    y_pred = forward(X)
    l = loss(Y,y_pred)
    dw = gradient(X,Y,y_pred)
    
    #update weights
    w-=learning_rate*dw

    if epoch%10==0:
         print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}')
      
print(f'Prediction after training:f(5)={forward(5):.3f}')
             

Prediction before training:f(5)=0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506905
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training:f(5)=10.000


STEP - 02
 - prediction = manually
 -  gradient computations = Autograd
 -  Loss computation = manually
 -  Parameter updates = manually

In [21]:
import torch

# Here we replace the manually computed gradient with autograd

X = torch.tensor([1,2,3,4], dtype = torch.float32)
Y = torch.tensor([2,4,6,8], dtype = torch.float32)

w = torch.tensor(0.0, dtype = torch.float32, requires_grad = True)

# model output
def forward(x):
    return w*x
    
# loss = MSE
def loss(y,y_pred):
    return ((y_pred - y)**2).mean()

print(f'Prediction before training : f(5)= {forward(5):.3f}')

#Training
learning_rate = 0.01
n_iters = 100

for epoch in range(n_iters):
    
    # predict = forward pass
    y_pred = forward(X)

    l = loss(Y,y_pred)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    #w.data = w.data - learning_rate * w.grad
    with torch.no_grad():
        w-=learning_rate*w.grad

    #zero the gradients after updating
    w.grad.zero_()

    if epoch % 10 == 0:
        print(f'epoch {epoch+1}: w = {w.item():.3f}, loss = {l.item():.8f}')

print(f'Prediction after training: f(5) = {forward(5).item():.3f}')
    
    


Prediction before training : f(5)= 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000


STEP - 03
- prediction = manually
- gradient computations = Autograd
- Loss computation = PyTorch loss
- Parameter updates = PyTorch optimiser

In [22]:
# Design model (input, output, forward pass with different layers)
# 2) Construct loss and optimizer
# 3) Training loop
#       - Forward = compute prediction and loss
#       - Backward = compute gradients
#       - Update weights

import torch
import torch.nn as nn

# 0) Training samples
X = torch.tensor([1,2,3,4], dtype = torch.float32)
Y = torch.tensor([2,4,6,8], dtype = torch.float32)


# 1) Design Model: Weights to optimize and forward function
w = torch.tensor(0.0, dtype = torch.float32, requires_grad = True)

def forward(x):
    return w*x

print(f'Prediction before training : f(5) = {forward(5) : .3f}')

# 2) Define loss and optimizer
learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()

optimizer = torch.optim.SGD([w], lr = learning_rate)

# 3) Training loop
for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    #loss
    l = loss(Y, y_pred)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

     # zero the gradients after updating
    optimizer.zero_grad()

    if epoch%10==0:
        print('epoch',epoch+1 ,': w=',w , ':l=',l)

print(f'Prediction after training: f(5) = {forward(5).item():.3f}')
    

Prediction before training : f(5) =  0.000
epoch 1 : w= tensor(0.3000, requires_grad=True) :l= tensor(30., grad_fn=<MseLossBackward0>)
epoch 11 : w= tensor(1.6653, requires_grad=True) :l= tensor(1.1628, grad_fn=<MseLossBackward0>)
epoch 21 : w= tensor(1.9341, requires_grad=True) :l= tensor(0.0451, grad_fn=<MseLossBackward0>)
epoch 31 : w= tensor(1.9870, requires_grad=True) :l= tensor(0.0017, grad_fn=<MseLossBackward0>)
epoch 41 : w= tensor(1.9974, requires_grad=True) :l= tensor(6.7705e-05, grad_fn=<MseLossBackward0>)
epoch 51 : w= tensor(1.9995, requires_grad=True) :l= tensor(2.6244e-06, grad_fn=<MseLossBackward0>)
epoch 61 : w= tensor(1.9999, requires_grad=True) :l= tensor(1.0176e-07, grad_fn=<MseLossBackward0>)
epoch 71 : w= tensor(2.0000, requires_grad=True) :l= tensor(3.9742e-09, grad_fn=<MseLossBackward0>)
epoch 81 : w= tensor(2.0000, requires_grad=True) :l= tensor(1.4670e-10, grad_fn=<MseLossBackward0>)
epoch 91 : w= tensor(2.0000, requires_grad=True) :l= tensor(5.0768e-12, grad_

STEP - 04
- prediction = PyTorch model
-  gradient computations = Autograd
-   Loss computation = PyTorch loss
-    Parameter updates = PyTorch optimiser

In [23]:
# 1) Design model (input, output, forward pass with different layers)
# 2) Construct loss and optimizer
# 3) Training loop
#       - Forward = compute prediction and loss
#       - Backward = compute gradients
#       - Update weights

import torch
import torch.nn as nn

# Linear regression
# f = w * x 

# here : f = 2 * x

# 0) Training samples, watch the shape!
X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

n_samples, n_features = X.shape
print(f'#samples: {n_samples}, #features: {n_features}')
# 0) create a test sample
X_test = torch.tensor([5], dtype=torch.float32)

# 1) Design Model, the model has to implement the forward pass!
# Here we can use a built-in model from PyTorch
input_size = n_features
output_size = n_features

# we can call this model with samples X
model = nn.Linear(input_size, output_size)

'''
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define diferent layers
        self.lin = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.lin(x)

model = LinearRegression(input_size, output_size)
'''

print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# 2) Define loss and optimizer
learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop
for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X)

    # loss
    l = loss(Y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()

    if epoch % 10 == 0:
        [w, b] = model.parameters() # unpack parameters
        print('epoch ', epoch+1, ': w = ', w[0][0].item(), ' loss = ', l)

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

#samples: 4, #features: 1
Prediction before training: f(5) = 2.630
epoch  1 : w =  0.8926337957382202  loss =  tensor(18.3799, grad_fn=<MseLossBackward0>)
epoch  11 : w =  1.8724561929702759  loss =  tensor(0.4807, grad_fn=<MseLossBackward0>)
epoch  21 : w =  2.0285654067993164  loss =  tensor(0.0173, grad_fn=<MseLossBackward0>)
epoch  31 : w =  2.0522260665893555  loss =  tensor(0.0051, grad_fn=<MseLossBackward0>)
epoch  41 : w =  2.0546247959136963  loss =  tensor(0.0045, grad_fn=<MseLossBackward0>)
epoch  51 : w =  2.053645133972168  loss =  tensor(0.0042, grad_fn=<MseLossBackward0>)
epoch  61 : w =  2.0521621704101562  loss =  tensor(0.0040, grad_fn=<MseLossBackward0>)
epoch  71 : w =  2.05063796043396  loss =  tensor(0.0037, grad_fn=<MseLossBackward0>)
epoch  81 : w =  2.049144744873047  loss =  tensor(0.0035, grad_fn=<MseLossBackward0>)
epoch  91 : w =  2.0476934909820557  loss =  tensor(0.0033, grad_fn=<MseLossBackward0>)
Prediction after training: f(5) = 10.096
